<a href="https://colab.research.google.com/github/sondosa20/Trying-T5-in-event-Prediction/blob/main/Copie_de_T5ForPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


# Load my csv file
log_file = "/content/data.csv"
data = pd.read_csv(log_file)
#print(data.head())

# Sort by UserID and Timestamp
data = data.sort_values(by=["UserID", "Timestamp"])
#print(data.head(10))

#Create Sequences: Group events by UserID, and for each user, create sequences of events leading to the next one.
sequences = []

# Grouper les données par UserID
for user_id, group in data.groupby("UserID"):
    # Extraire les événements pour chaque user
    events = group["StudentEvent"].tolist()

    # Construire des séquences Input et Output
    for i in range(len(events) - 1):
        input_sequence = " → ".join(events[:i + 1])
        output_event = events[i + 1]
        sequences.append({"Input": input_sequence, "Output": output_event})


# Créer un DataFrame pour les séquences
processed_data = pd.DataFrame(sequences)

# Vérifier
#print(processed_data.head())
# Sauvegarder dans un fichier CSV
processed_data.to_csv("processed_data.csv", index=False)

print("Les données transformées ont été sauvegardées dans 'processed_data.csv'")

Les données transformées ont été sauvegardées dans 'processed_data.csv'


In [ ]:
#Split dataset into training and testing sets
train_data, test_data  = train_test_split(pd.read_csv("processed_data.csv"), test_size=0.2, random_state=42)

# Save the splits
train_data.to_csv("train_data.csv", index=False)
test_data.to_csv("test_data.csv", index=False)

print(f"Nombre d'exemples d'entraînement : {len(train_data)}")
print(f"Nombre d'exemples de test : {len(test_data)}")

Nombre d'exemples d'entraînement : 24329
Nombre d'exemples de test : 6083


In [ ]:
!pip install datasets transformers

from datasets import load_dataset
from transformers import T5Tokenizer

# Load train and test datasets
dataset = load_dataset("csv", data_files={
    "train": "train_data.csv",
    "test": "test_data.csv"
})


# Load T5 tokenizer pour convertir text rn tockens numeriques
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Fonction de prétraitement pour tokeniser les données
def preprocess_data(examples):
    inputs = examples["Input"]
    targets = examples["Output"]

    # Tokeniser les inputs et outputs
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    # Ajouter les labels au modèle
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# Apply preprocessing to the datasets
tokenized_train = dataset["train"].map(preprocess_data, batched=True)
tokenized_test = dataset["test"].map(preprocess_data, batched=True)

# Vérifier un exemple tokenisé
print("Exemple tokenisé (train) :", tokenized_train[0])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/24329 [00:00<?, ? examples/s]

Map:   0%|          | 0/6083 [00:00<?, ? examples/s]

Exemple tokenisé (train) : {'Input': 'viewed_course_core → viewed_course_core → viewed_course_core → viewed_course_core → viewed_course_core → viewed_course_module_mod_resource → viewed_course_core → viewed_course_core → viewed_course_module_mod_resource → viewed_course_module_mod_resource → viewed_course_module_mod_resource → viewed_course_module_mod_resource → viewed_course_module_mod_url → viewed_course_core → viewed_course_core → viewed_course_module_mod_resource → viewed_course_module_mod_resource → viewed_course_module_mod_resource → viewed_course_module_mod_resource → viewed_course_module_mod_resource → viewed_course_module_mod_resource → viewed_course_module_mod_resource → viewed_course_module_mod_resource → viewed_course_core → viewed_course_module_mod_resource → viewed_course_module_mod_resource → viewed_course_module_mod_resource → viewed_course_module_mod_resource → viewed_course_module_mod_resource → viewed_course_module_mod_resource → viewed_course_module_mod_resource → v

In [ ]:
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer

# Charger le modèle T5 pré-entraîné
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Configurer les arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"
)

# Initialiser le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
)

# Entraîner le modèle
trainer.train()


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-4-0fa7973f2254>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.030300,0.024019
2,0.026200,0.020648
3,0.026800,0.019796


TrainOutput(global_step=4563, training_loss=0.11245798937765726, metrics={'train_runtime': 3382.9762, 'train_samples_per_second': 21.575, 'train_steps_per_second': 1.349, 'total_flos': 9878192064036864.0, 'train_loss': 0.11245798937765726, 'epoch': 3.0})

In [ ]:
# Évaluer le modèle
eval_results = trainer.evaluate()
print("Résultats de l'évaluation :", eval_results)

Résultats de l'évaluation : {'eval_loss': 0.019795842468738556, 'eval_runtime': 86.5488, 'eval_samples_per_second': 70.284, 'eval_steps_per_second': 4.402, 'epoch': 3.0}


In [ ]:
# Sauvegarder le modèle et le tokenizer
model.save_pretrained("./fine_tuned_t5")
tokenizer.save_pretrained("./fine_tuned_t5")

print("Modèle et tokenizer sauvegardés dans './fine_tuned_t5'")

Modèle et tokenizer sauvegardés dans './fine_tuned_t5'


In [ ]:
#Génerer les prédictions
import torch
from transformers import T5ForConditionalGeneration

# Charger le modèle entraîné
model = T5ForConditionalGeneration.from_pretrained("./fine_tuned_t5")

# Déplacer le modèle sur l'appareil (GPU ou CPU)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(device)

# Générer des prédictions
predictions = []
true_outputs = []

for example in tokenized_test:
    # Préparer l'entrée
    input_ids = torch.tensor(example["input_ids"]).unsqueeze(0).to(device)

    # Générer la prédiction
    outputs = model.generate(input_ids, max_length=128)
    predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Ajouter les prédictions et les vérités à leurs listes
    predictions.append(predicted_text)
    true_outputs.append(tokenizer.decode(example["labels"], skip_special_tokens=True))

In [ ]:
test_dataset = dataset["test"]
for i in range(5):
    print(f"Input : {test_dataset[i]['Input']}")
    print(f"Vérité terrain : {true_outputs[i]}")
    print(f"Prédiction : {predictions[i]}")
    print("-" * 50)

Input : viewed_course_core → viewed_course_module_mod_resource → viewed_course_module_mod_resource → viewed_course_core → viewed_course_core → viewed_course_module_mod_forum → viewed_discussion_mod_forum → uploaded_assessable_mod_forum → created_discussion_subscription_mod_forum → created_post_mod_forum → viewed_discussion_mod_forum → viewed_course_core → viewed_course_module_mod_forum → viewed_discussion_mod_forum → uploaded_assessable_mod_forum → created_discussion_subscription_mod_forum → created_post_mod_forum → viewed_discussion_mod_forum → viewed_course_core → viewed_course_module_mod_forum → viewed_discussion_mod_forum → viewed_course_core → viewed_course_module_mod_forum → viewed_discussion_mod_forum → viewed_discussion_mod_forum → viewed_discussion_mod_forum → uploaded_assessable_mod_forum → created_discussion_subscription_mod_forum → created_post_mod_forum → viewed_discussion_mod_forum → uploaded_assessable_mod_forum → created_discussion_subscription_mod_forum → created_post_

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(true_outputs, predictions, average="weighted")
recall = recall_score(true_outputs, predictions, average="weighted")
f1 = f1_score(true_outputs, predictions, average="weighted")

print(f"Précision : {precision * 100:.2f}%")
print(f"Rappel : {recall * 100:.2f}%")
print(f"F1-score : {f1 * 100:.2f}%")

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Précision : 26.41%
Rappel : 25.23%
F1-score : 14.10%


In [ ]:
#Sauvegarde les prédictions
import pandas as pd

results_df = pd.DataFrame({
    "Input": [test_dataset[i]["Input"] for i in range(len(predictions))],
    "Vérité Terrain": true_outputs,
    "Prédiction": predictions
})
results_df.to_csv("evaluation_results.csv", index=False)
print("Les résultats ont été sauvegardés dans 'evaluation_results.csv'.")

Les résultats ont été sauvegardés dans 'evaluation_results.csv'.
